In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)


In [2]:
hbo = pd.read_pickle("hbo_ids_final.pkl")

In [3]:
hbo.head()

,show,year,rating,imdb,rotten_tomatoes,tconst,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId,ordering,title,region,language,isOriginalTitle,imdb_titles,titleType
0,Game of Thrones,2011,18+,9.3,89%,tt0944947,Game of Thrones,Game of Thrones,0.0,2011.0,2011.0,57,"Action,Adventure,Drama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Wire,2002,18+,9.3,94%,tt0306414,The Wire,The Wire,0.0,2002.0,2002.0,59,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chernobyl,2019,18+,9.4,96%,tt7366338,Chernobyl,Chernobyl,0.0,2019.0,2019.0,330,"Drama,History,Thriller",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,The Sopranos,1999,18+,9.2,92%,tt0141842,The Sopranos,The Sopranos,0.0,1999.0,1999.0,55,"Crime,Drama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Band of Brothers,2001,18+,9.4,94%,tt0185906,Band of Brothers,Band of Brothers,0.0,2001.0,2001.0,594,"Action,Drama,History",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
hbo["tconst"] = np.where(hbo["tconst"].isna(), hbo["titleId"], hbo["tconst"])

In [5]:
hbo[hbo["tconst"].isna()]

,show,year,rating,imdb,rotten_tomatoes,tconst,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,titleId,ordering,title,region,language,isOriginalTitle,imdb_titles,titleType


In [6]:
hbo = hbo[["show", "year", "rating", "imdb", "rotten_tomatoes", "tconst"]].copy()

In [7]:
imdb = pd.read_pickle("imdb_tv_all.pkl")

In [8]:
hbo = hbo.merge(imdb, how="left", on="tconst")

In [10]:
hbo.head()

,show,year,rating,imdb,rotten_tomatoes,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,Game of Thrones,2011,18+,9.3,89%,tt0944947,tvSeries,Game of Thrones,Game of Thrones,0.0,2011.0,2011.0,57,"Action,Adventure,Drama"
1,The Wire,2002,18+,9.3,94%,tt0306414,tvSeries,The Wire,The Wire,0.0,2002.0,2002.0,59,"Crime,Drama,Thriller"
2,Chernobyl,2019,18+,9.4,96%,tt7366338,tvMiniSeries,Chernobyl,Chernobyl,0.0,2019.0,2019.0,330,"Drama,History,Thriller"
3,The Sopranos,1999,18+,9.2,92%,tt0141842,tvSeries,The Sopranos,The Sopranos,0.0,1999.0,1999.0,55,"Crime,Drama"
4,Band of Brothers,2001,18+,9.4,94%,tt0185906,tvMiniSeries,Band of Brothers,Band of Brothers,0.0,2001.0,2001.0,594,"Action,Drama,History"


In [12]:
hbo.dtypes

show                object
year                 int64
rating              object
imdb               float64
rotten_tomatoes     object
tconst              object
titleType           object
primaryTitle        object
originalTitle       object
isAdult            float64
startYear          float64
endYear            float64
runtimeMinutes      object
genres              object
dtype: object

In [13]:
hbo["genres"] = hbo["genres"].astype("str")

In [14]:
def unique_genres(df):
    
    # get list of unique genres
    genres = [genre.split(",") for genre in df["genres"]]
    uniques = []

    for i in genres:
        for j in i:
            if j not in uniques:
                uniques.append(j)
    return uniques

In [16]:
uniques = unique_genres(hbo)

In [17]:
uniques

['Action',
 'Adventure',
 'Drama',
 'Crime',
 'Thriller',
 'History',
 'Mystery',
 'Sci-Fi',
 'Comedy',
 'News',
 'Talk-Show',
 'Fantasy',
 'Music',
 'Musical',
 'Documentary',
 'Family',
 'War',
 'Sport',
 'Biography',
 'Romance',
 'Animation',
 'Horror',
 '\\N',
 'Reality-TV',
 'Game-Show',
 'nan']

In [18]:
uniques.remove("\\N")
uniques.remove("nan")

In [19]:
def genres(df, uniques):
    
    # create columns for each genre
    for genre in uniques:
        df[genre] = 0
    
    # place 1 if tv show has that genre
    for genre in uniques:
        df[genre] = df["genres"].str.contains(genre).astype(int)
        
    return df

In [21]:
hbo = genres(hbo, uniques)

In [22]:
hbo.head()

,show,year,rating,imdb,rotten_tomatoes,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,Action,Adventure,Drama,Crime,Thriller,History,Mystery,Sci-Fi,Comedy,News,Talk-Show,Fantasy,Music,Musical,Documentary,Family,War,Sport,Biography,Romance,Animation,Horror,Reality-TV,Game-Show
0,Game of Thrones,2011,18+,9.3,89%,tt0944947,tvSeries,Game of Thrones,Game of Thrones,0.0,2011.0,2011.0,57,"Action,Adventure,Drama",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Wire,2002,18+,9.3,94%,tt0306414,tvSeries,The Wire,The Wire,0.0,2002.0,2002.0,59,"Crime,Drama,Thriller",0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Chernobyl,2019,18+,9.4,96%,tt7366338,tvMiniSeries,Chernobyl,Chernobyl,0.0,2019.0,2019.0,330,"Drama,History,Thriller",0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Sopranos,1999,18+,9.2,92%,tt0141842,tvSeries,The Sopranos,The Sopranos,0.0,1999.0,1999.0,55,"Crime,Drama",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Band of Brothers,2001,18+,9.4,94%,tt0185906,tvMiniSeries,Band of Brothers,Band of Brothers,0.0,2001.0,2001.0,594,"Action,Drama,History",1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
# hbo.to_pickle("hbo_genres.pkl")

In [24]:
hbo_genres = pd.DataFrame(hbo[uniques].sum().sort_values(ascending=False))

In [25]:
hbo_genres

,0
Drama,92
Comedy,69
Crime,28
Documentary,19
Action,14
Mystery,14
Romance,13
History,13
Sport,10
Adventure,9


In [31]:
hbo_genres["ratio"] = round((hbo_genres[0] / len(hbo)), 4)

In [32]:
hbo_genres

,0,ratio
Drama,92,0.4868
Comedy,69,0.3651
Crime,28,0.1481
Documentary,19,0.1005
Action,14,0.0741
Mystery,14,0.0741
Romance,13,0.0688
History,13,0.0688
Sport,10,0.0529
Adventure,9,0.0476
